# 📦 서울시 지하철 관리자 모니터링 시스템 - API POC
이 노트북은 서울시 공공 API를 활용하여 혼잡도, 실시간 도착 정보, 공기질, 기상 정보를 수집하는 POC입니다.
Python `requests` 라이브러리를 활용합니다.

In [ ]:
# ✅ 공통 준비
import requests
import urllib.parse
from datetime import datetime

# API 인증키 (인코딩키 권장)
API_KEY = '45494d657a716f7235385773656170'

## 🚇 1. 서울교통공사_지하철 혼잡도 정보

In [48]:
import requests

import json

# 1. API 인증키
API_KEY = "nRSQWm1z9nVhB2zcUdDbSXX3QXQhfdYrq6+d4GWM7hknETE77/J0TYLZ6Nu5arvwmDkAipXwWiKPs4orwkfMmw=="  # 반드시 본인의 인코딩된 인증키를 넣어주세요


# 2. Base URL 및 엔드포인트 설정
base_url = "https://api.odcloud.kr/api"
service_path = "/15071311/v1/uddi:7bd50077-dea4-48c5-a50f-c1f073afcf1e"
url = base_url + service_path

# 3. 파라미터 설정 (잠실역 기준)
params = {
    "page": 1,
    "perPage": 1700,
    "serviceKey": API_KEY,
    "data":"2025-06-30"
}

# 4. 요청 및 응답
response = requests.get(url, params=params)

# 5. 응답 확인 및 저장
try:
    data = response.json()
    
    # data 필드가 있는지 확인하고 저장
    if 'data' in data:
        with open("잠실역_혼잡도0630.json", "w", encoding="utf-8") as f:
            json.dump(data['data'], f, ensure_ascii=False, indent=2)
        print("✅ 저장 완료: 잠실역_혼잡도0630.json")
    else:
        print("⚠️ 'data' 필드 없음:", data)
except Exception as e:
    print("❌ 오류 발생:", e)
    print("응답 내용:", response.text)

✅ 저장 완료: 잠실역_혼잡도0630.json


## 🕒 2. 서울교통공사_실시간 지하철 도착 정보

In [86]:
import requests
import json

# API 정보
API_KEY = "4d71567967716f7237344e68777546"
line = "금정"
url = f"http://swopenapi.seoul.go.kr/api/subway/4d71567967716f7237344e68777546/json/realtimeStationArrival/1/5/수리산"

# 요청
response = requests.get(url)

# 응답 디코딩 및 파싱
try:
    # 디코딩 (한글 깨짐 방지)
    decoded_text = response.content.decode('utf-8')
    
    # JSON 파싱
    data = json.loads(decoded_text)

    # 사람이 보기 쉽게 출력
    print(json.dumps(data, indent=2, ensure_ascii=False))

    # JSON 파일 저장
    with open("실시간지하철_1호선_금정역.json", "w", encoding="utf-8") as f:
        json.dump(data, f, indent=2, ensure_ascii=False)

    print("✅ JSON 파일로 저장 완료")

except Exception as e:
    print("❌ 오류 발생:", e)
    print("원본 응답:", response.text)

{
  "errorMessage": {
    "status": 200,
    "code": "INFO-000",
    "message": "정상 처리되었습니다.",
    "link": "",
    "developerMessage": "",
    "total": 4
  },
  "realtimeArrivalList": [
    {
      "beginRow": null,
      "endRow": null,
      "curPage": null,
      "pageRow": null,
      "totalCount": 4,
      "rowNum": 1,
      "selectedCount": 4,
      "subwayId": "1004",
      "subwayNm": null,
      "updnLine": "상행",
      "trainLineNm": "불암산행 - 산본방면",
      "subwayHeading": null,
      "statnFid": "1004000446",
      "statnTid": "1004000444",
      "statnId": "1004000445",
      "statnNm": "수리산",
      "trainCo": null,
      "trnsitCo": "1",
      "ordkey": "01001불암산0",
      "subwayList": "1004",
      "statnList": "1004000445",
      "btrainSttus": "일반",
      "barvlDt": "0",
      "btrainNo": "4596",
      "bstatnId": "1004000409",
      "bstatnNm": "불암산",
      "recptnDt": "2025-07-02 12:59:04",
      "arvlMsg2": "전역 도착",
      "arvlMsg3": "대야미",
      "arvlCd": "5",
      "l

## 🌫 3. 서울교통공사_역사 내 공기질 정보

In [93]:
import requests
import xml.etree.ElementTree as ET
import json

# API 키와 지하철역 코드 (잠실역: 261)
API_KEY = "45494d657a716f7235385773656170"
stationCd = 261  # 잠실역

# API 요청 URL 구성
url = f"http://openapi.seoul.go.kr:8088/45494d657a716f7235385773656170/xml/airPolutionInfo/1/250/261"


# API 요청
response = requests.get(url)

# XML 파싱
root = ET.fromstring(response.content)

# 데이터 저장용 리스트
data_list = []

# 각 항목 반복
for row in root.iter("row"):
    item = {}
    for child in row:
        item[child.tag] = child.text
    data_list.append(item)

# JSON 파일로 저장
with open("station_air_info.json", "w", encoding="utf-8") as f:
    json.dump(data_list, f, ensure_ascii=False, indent=4)


## ⛅ 4. 기상청_단기예보 조회 서비스 (잠실 기준)

In [ ]:
import requests
import xml.etree.ElementTree as ET
import csv
from datetime import datetime, timedelta, timezone

# 현재 시간 (KST 기준)
now_kst = datetime.now(timezone.utc) + timedelta(hours=9)
base_date = now_kst.strftime('%Y%m%d')
adjusted_hour = now_kst.replace(minute=0, second=0, microsecond=0) - timedelta(hours=1)
base_time = adjusted_hour.strftime('%H%M')

# 서울 중구 (nx=60, ny=127)
nx, ny = 60, 127

API_KEY = "nRSQWm1z9nVhB2zcUdDbSXX3QXQhfdYrq6%2Bd4GWM7hknETE77%2FJ0TYLZ6Nu5arvwmDkAipXwWiKPs4orwkfMmw%3D%3D"

url = (
    f"http://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getUltraSrtFcst"
    f"?serviceKey={API_KEY}"
    f"&pageNo=1&numOfRows=1000"
    f"&dataType=XML"
    f"&base_date={base_date}&base_time={base_time}"
    f"&nx={nx}&ny={ny}"
)

response = requests.get(url)
root = ET.fromstring(response.content)

# 필터: 강수량(RN1), 강수형태(PTY)
rows = []
for item in root.iter("item"):
    category = item.find("category").text
    if category in ["RN1", "PTY"]:
        fcst_date = item.find("fcstDate").text
        fcst_time = item.find("fcstTime").text
        fcst_value = item.find("fcstValue").text
        rows.append([category, fcst_date, fcst_time, fcst_value])

# CSV 저장
filename = f"rain_snow_fcst_{base_date}.csv"
with open(filename, 'w', newline='', encoding='utf-8-sig') as f:
    writer = csv.writer(f)
    writer.writerow(["category", "fcstDate", "fcstTime", "fcstValue"])
    writer.writerows(rows)

print(f"✅ 비/눈 관련 예보만 CSV로 저장 완료: {filename}")

✅ 초단기 예보 데이터 저장 완료: seoul_ultra_fcst_20250702.json
